In [1]:
import numpy as np
def erb_space(low_freq=50, high_freq=8000, n=64):
    ear_q = 9.26449
    min_bw = 24.7

    cf_array = -(ear_q * min_bw) + np.exp(np.linspace(1,n,n) * (-np.log(high_freq + ear_q * min_bw) + np.log(low_freq + ear_q * min_bw)) / n) \
                * (high_freq + ear_q * min_bw)
    return cf_array

In [8]:
def make_erb_filters(sr, num_channels, low_freq):
    t = 1. / sr
    cf = erb_space(low_freq, sr // 2, num_channels)

    ear_q = 9.26449
    min_bw = 24.7
    order = 4

    erb = np.power(np.power(cf/ear_q, order) + (min_bw ** order), 1. / order)
    b = 1.019 * 2 * np.pi * erb

    a0 = t
    a2 = 0
    b0 = 1
    b1 = -2 * np.cos(2 * cf * np.pi * t) / np.exp(b*t)
    b2 = np.exp(-2 * b * t)

    a11 = -(2 * t * np.cos(2*cf*np.pi*t) / np.exp(b*t) + 2 * np.sqrt(3+2**1.5) * t * np.sin(2*cf*np.pi*t) / np.exp(b*t))/2
    a12 = -(2 * t * np.cos(2*cf*np.pi*t) / np.exp(b*t) - 2 * np.sqrt(3+2**1.5) * t * np.sin(2*cf*np.pi*t) / np.exp(b*t))/2
    a13 = -(2 * t * np.cos(2*cf*np.pi*t) / np.exp(b*t) + 2 * np.sqrt(3-2**1.5) * t * np.sin(2*cf*np.pi*t) / np.exp(b*t))/2
    a14 = -(2 * t * np.cos(2*cf*np.pi*t) / np.exp(b*t) - 2 * np.sqrt(3-2**1.5) * t * np.sin(2*cf*np.pi*t) / np.exp(b*t))/2

    p1 = (-2*np.exp(4j*cf*np.pi*t)*t + 2*np.exp(-(b*t) + 2j*cf*np.pi*t) * t *
         (np.cos(2*cf*np.pi*t) - np.sqrt(3 - 2**(3/2))* np.sin(2*cf*np.pi*t)))
    p2 = (-2*np.exp(4j*cf*np.pi*t)*t + 2*np.exp(-(b*t) + 2j*cf*np.pi*t) * t *
         (np.cos(2*cf*np.pi*t) + np.sqrt(3 - 2**(3/2))* np.sin(2*cf*np.pi*t)))
    p3 = (-2*np.exp(4j*cf*np.pi*t)*t + 2*np.exp(-(b*t) + 2j*cf*np.pi*t) * t *
         (np.cos(2*cf*np.pi*t) - np.sqrt(3 + 2**(3/2))* np.sin(2*cf*np.pi*t)))
    p4 = (-2*np.exp(4j*cf*np.pi*t)*t + 2*np.exp(-(b*t) + 2j*cf*np.pi*t) * t *
         (np.cos(2*cf*np.pi*t) + np.sqrt(3 + 2**(3/2))* np.sin(2*cf*np.pi*t)))
    p5 = np.power(-2 / np.exp(2*b*t) - 2 * np.exp(4j*cf*np.pi*t) + 2 * (1 + np.exp(4j*cf*np.pi*t)) / np.exp(b*t), 4)
    gain = np.abs(p1 * p2 * p3 * p4 / p5)

    allfilts = np.ones((np.size(cf, 0), 1), dtype=np.float32)
    fcoefs = np.column_stack((a0*allfilts, a11, a12, a13, a14, a2*allfilts, b0*allfilts, b1, b2, gain))
    return fcoefs, cf

In [11]:
make_erb_filters(8000,23,50)

(array([[ 1.25000006e-04,  5.38739065e-06,  1.66433136e-04,
          7.20947225e-05,  9.97258040e-05,  0.00000000e+00,
          1.00000000e+00,  1.37456421e+00,  5.43554583e-01,
          2.50365635e-14],
        [ 1.25000006e-04, -7.53772814e-05,  2.21623307e-04,
          4.76443905e-05,  9.86016354e-05,  0.00000000e+00,
          1.00000000e+00,  1.16996821e+00,  5.84355710e-01,
          3.95738155e-14],
        [ 1.25000006e-04, -1.45384317e-04,  2.53387690e-04,
          1.97924567e-05,  8.82109166e-05,  0.00000000e+00,
          1.00000000e+00,  8.64026987e-01,  6.23169524e-01,
          6.20576142e-14],
        [ 1.25000006e-04, -2.00860978e-04,  2.64612833e-04,
         -8.05541268e-06,  7.18072674e-05,  0.00000000e+00,
          1.00000000e+00,  5.10014838e-01,  6.59812561e-01,
          9.83267935e-14],
        [ 1.25000006e-04, -2.41236368e-04,  2.59656194e-04,
         -3.37598752e-05,  5.21797018e-05,  0.00000000e+00,
          1.00000000e+00,  1.47358613e-01,  6.941725